In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
workdir_path = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification'
os.chdir(workdir_path)
!ls

In [ ]:
import torch
import torchvision
import pandas as pd
import tensorflow as tf
from PIL import Image
import numpy as np
import collections
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [ ]:
sorted(os.listdir('train/00000'))

In [ ]:
# Declaração de constantes
TRAIN_FOLDER = 'train'
TEST_FOLDER = 'test'
mri_types = collections.namedtuple('mri_types', ['FLAIR', 'T1W', 'T1WCE', 'T2W'])
MRI_TYPES = mri_types('FLAIR', 'T1w', 'T1wCE', 'T2w')

### TODO-Criar bloco com funções acessórias

In [ ]:
def loader(folder, patient_id, mri):
    for files in os.listdir(os.path.join(folder, patient_id, mri)):
        print(patient_id, files)

In [ ]:
''' Função para ler todas as imagens de cada paciente
Estimativa de 17 horas para ler todos os arquivos. Necessário criar outra
função

def image_loader(folder:str):
  for dirpath, dirnames, filenames in os.walk(folder):
    for i in range(0, len(dirnames)):
      for patient, empty, photos in os.walk(os.path.join(dirpath, dirnames[i])):
        for files in photos:
          print('faltam', len(dirnames)-i, 'pacientes' )
          print('faltam ', len(photos)-t, 'fotos na pasta', patient )
          file_path = os.path.join(patient, files)
          try:
            yield file_path
            im.close()
          except ValueError as err:
            print(err)
          except TypeError as err:
            print (err)
          except AttributeError as err:
            print(err)
'''

## TODO-Análise exploratória do dataset

In [ ]:
df_label = pd.read_csv("train_labels.csv", dtype={'BraTS21ID':str, 'MGMT_value':int})

In [ ]:
df_label.MGMT_value.unique()

In [ ]:
df_label.describe()

In [ ]:
df_label.head()

In [ ]:
# Lista os pacientes que fazem parte do diretório de treinamento
train_patients = [subdirs for subdirs in os.listdir(TRAIN_FOLDER)]
print('Número de pacientes no diretório de treino', len(train_patients))

# Lista os pacientes que fazem parte do diretório de teste
test_patients = [subdirs for subdirs in os.listdir(TEST_FOLDER)]
print('Número de pacientes no diretório de teste', len(test_patients))

print('Total de pacientes', len(test_patients)+len(train_patients))

### Função de exclusão de casos com falha no dataset

In [ ]:
# Exclusão dos pacientes 00109, 00123, 00709 devido a falha do dataset
patients_delete = ('00109', '00123', '00709')
try:
    for patient in patients_delete:
        df_label = df_label[df_label.BraTS21ID != patient]
        train_patients.remove(patient)
except Exception as err:
    print('erro: ', err)
print('Número de pacientes no diretório de treino', len(train_patients))

In [ ]:
def train_path(patient):
    for mri in MRI_TYPES:
        path = [TRAIN_FOLDER, 
                patient, 
                mri, 
                sorted(os.listdir(os.path.join(TRAIN_FOLDER, patient, mri)), 
                       key=lambda file: int(re.sub('[^0-9]', '', file)))
               ]
        print(path)
    return path

In [ ]:
train_patients = sorted(train_patients)
train_patients

Após ordenação dos pacientes e a ordenação correta dos arquivos de imagem, aparentemente o código abaixo funcionou mais rápido.

In [ ]:
image_path = list(map(train_path, train_patients))

## TODO - Ordenar caminhos das imagens para otimização.
Fazer através de dataframe ou alterando a linha anterior para loop for no lugar de map

In [ ]:
sorted(image_path, key=lambda x: x[2])

In [ ]:
# Após verificar que as imagens não estavam ordenadas corretamente, fiz testes
# até chegar na função lambda abaixo para ordenar de forma correta.
# inserido na definição da função train_path()
#sorted(image_path[0][3], key=lambda number: int(re.sub('[^0-9]', '', number)))